In [1]:
from __future__ import print_function

from miscc.utils import mkdir_p
from miscc.utils import build_super_images
from miscc.losses import sent_loss, words_loss
from miscc.config import cfg, cfg_from_file

from datasets import TextDataset
from datasets import prepare_data

from model import RNN_ENCODER, CNN_ENCODER

import os
import sys
import time
import random
import pprint
import datetime
import dateutil.tz
import argparse
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms

In [2]:
# dataset.n_words

In [3]:
cfg.TEXT.EMBEDDING_DIM

256

In [4]:
def build_models():
    # build model ############################################################
    text_encoder = RNN_ENCODER(dataset.n_words, nhidden=cfg.TEXT.EMBEDDING_DIM)
    cfg.TREE.BASE_SIZE = torch.FloatTensor(299)
#     image_encoder = CNN_ENCODER(cfg.TREE.BASE_SIZE) #299 inside CNN_Encoder
    labels = torch.LongTensor(range(batch_size))
    start_epoch = 0
    if cfg.TRAIN.NET_E != '':
        state_dict = torch.load(cfg.TRAIN.NET_E)
        text_encoder.load_state_dict(state_dict)
        print('Load ', cfg.TRAIN.NET_E)
        #
        name = cfg.TRAIN.NET_E.replace('text_encoder')
        state_dict = torch.load(name)
#         image_encoder.load_state_dict(state_dict)
        print('Load ', name)

        istart = cfg.TRAIN.NET_E.rfind('_') + 8
        iend = cfg.TRAIN.NET_E.rfind('.')
        start_epoch = cfg.TRAIN.NET_E[istart:iend]
        start_epoch = int(start_epoch) + 1
        print('start_epoch', start_epoch)
    if cfg.CUDA:
        text_encoder = text_encoder
#         image_encoder = image_encoder
        labels = labels

    return text_encoder, labels, start_epoch

In [5]:
cfg.TEXT.EMBEDDING_DIM

256

In [6]:
print(torch.__version__)

0.4.0


In [7]:
#Configurations
cfg_from_file("cfg/DAMSM/coco.yml")
import pprint
pprint.pprint(cfg)
random.seed(random.randint(1, 10000))
np.random.seed(random.randint(1, 10000))
torch.manual_seed(random.randint(1, 10000))
if cfg.CUDA:
    torch.cuda.manual_seed_all(random.randint(1, 10000))
now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
output_dir = '../output/%s_%s_%s' % \
    (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

model_dir = os.path.join(output_dir, 'Model')
image_dir = os.path.join(output_dir, 'Image')
mkdir_p(model_dir)
mkdir_p(image_dir)
# Debug
torch.cuda.device(cfg.GPU_ID)
cudnn.benchmark = True

{'B_VALIDATION': False,
 'CONFIG_NAME': 'DAMSM',
 'CUDA': True,
 'DATASET_NAME': 'coco',
 'DATA_DIR': '../data/coco',
 'GAN': {'B_ATTENTION': True,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 64,
         'GF_DIM': 128,
         'R_NUM': 2,
         'Z_DIM': 100},
 'GPU_ID': 0,
 'RNN_TYPE': 'LSTM',
 'TEXT': {'CAPTIONS_PER_IMAGE': 5, 'EMBEDDING_DIM': 256, 'WORDS_NUM': 15},
 'TRAIN': {'BATCH_SIZE': 48,
           'B_NET_D': True,
           'DISCRIMINATOR_LR': 0.0002,
           'ENCODER_LR': 0.002,
           'FLAG': True,
           'GENERATOR_LR': 0.0002,
           'MAX_EPOCH': 600,
           'NET_E': '',
           'NET_G': '',
           'RNN_GRAD_CLIP': 0.25,
           'SMOOTH': {'GAMMA1': 4.0,
                      'GAMMA2': 5.0,
                      'GAMMA3': 10.0,
                      'LAMBDA': 1.0},
           'SNAPSHOT_INTERVAL': 5},
 'TREE': {'BASE_SIZE': 299, 'BRANCH_NUM': 1},
 'WORKERS': 1}


In [8]:
# cfg.TREE.BASE_SIZE = 299
imsize = cfg.TREE.BASE_SIZE * (2 ** (cfg.TREE.BRANCH_NUM-1))
batch_size = cfg.TRAIN.BATCH_SIZE
image_transform = transforms.Compose([
    transforms.Scale(int(imsize * 76 / 64)),
    transforms.RandomCrop(imsize),
    transforms.RandomHorizontalFlip()])
dataset = TextDataset("../data/coco", 'train',
                      base_size=299,
                      transform=image_transform)
print("=====================================================")
print("cfg.DATA_DIR: ", cfg.DATA_DIR)
print("cfg.TREE.BASE_SIZE: ", cfg.TREE.BASE_SIZE)
print("image_transform: ", image_transform)
print("cfg.TEXT.EMBEDDING_DIM", cfg.TEXT.EMBEDDING_DIM)
print("=====================================================")
print("dataset.n_words:", dataset.n_words, " dataset.embeddings_num: ", dataset.embeddings_num)
assert dataset
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, drop_last=True,
    shuffle=True, num_workers=int(cfg.WORKERS))

# # validation data #
dataset_val = TextDataset("../data/coco", 'test',
                          base_size=cfg.TREE.BASE_SIZE,
                          transform=image_transform)
dataloader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=batch_size, drop_last=True,
    shuffle=True, num_workers=int(cfg.WORKERS))

Load filenames from: ../data/coco/train/filenames.pickle (82783)
Load filenames from: ../data/coco/test/filenames.pickle (40470)
Load from:  ../data/coco/captions.pickle
cfg.DATA_DIR:  ../data/coco
cfg.TREE.BASE_SIZE:  299
image_transform:  <torchvision.transforms.Compose object at 0x1c17d7b400>
cfg.TEXT.EMBEDDING_DIM 256
dataset.n_words: 27297  dataset.embeddings_num:  5
Load filenames from: ../data/coco/train/filenames.pickle (82783)
Load filenames from: ../data/coco/test/filenames.pickle (40470)
Load from:  ../data/coco/captions.pickle


In [9]:
split_dir, bshuffle = 'train', True
if not cfg.TRAIN.FLAG:
    # bshuffle = False
    split_dir = 'test'
dataset = TextDataset(cfg.DATA_DIR, 'train',base_size=cfg.TREE.BASE_SIZE,transform=image_transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=cfg.TRAIN.BATCH_SIZE,drop_last=True, shuffle=bshuffle, num_workers=int(cfg.WORKERS))

Load filenames from: ../data/coco/train/filenames.pickle (82783)
Load filenames from: ../data/coco/test/filenames.pickle (40470)
Load from:  ../data/coco/captions.pickle


In [10]:
dataset

In [11]:
text_encoder, labels, start_epoch = build_models()

/Users/user/anaconda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [12]:
text_encoder.train()
text_encoder.eval()

RNN_ENCODER(
  (encoder): Embedding(27297, 300)
  (drop): Dropout(p=0.5)
  (rnn): LSTM(300, 128, batch_first=True, dropout=0.5, bidirectional=True)
)

In [13]:
for step, data in enumerate(dataloader, 0):
    imgs, captions, cap_lens, class_ids, keys = prepare_data(data)
    hidden = text_encoder.init_hidden(batch_size)
    print("step", step, "\ncap_lens\n", cap_lens, "\n") 
    if step == 0:
        break

step 0 
cap_lens
 tensor([ 15,  15,  14,  14,  13,  12,  12,  12,  11,  11,  10,  10,
         10,  10,  10,  10,  10,  10,  10,  10,   9,   9,   9,   9,
          9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,
          9,   9,   9,   8,   8,   8,   8,   8,   8,   8,   8,   7]) 



In [14]:
print(captions.size())
print(captions)
# captions_dummy_input = torch.rand(48, 15, requires_grad=True)
captions_fake_input = torch.randint(1, 23046, (48, 15), dtype=torch.long)
# captions_dummy_input = torch.rand(48, 15, requires_grad=True)
print(captions_fake_input.size())
print(captions_fake_input)

torch.Size([48, 15])
tensor([[ 23046,  24379,  10409,  23046,   7668,   8030,  18756,   7510,
          23501,   1943,  15355,    822,  25135,  10409,   1609],
        [ 23046,   7391,    335,   6301,  26444,   8494,    373,  23225,
           8707,  10400,  20140,  26444,  10402,    697,  25220],
        [  7016,  23543,   1943,  23046,   5937,   4530,  23046,  21520,
           3646,  24983,  10409,  23046,   4967,  27198,      0],
        [  8707,  26008,  11249,  10409,   8707,   6609,  12624,  26444,
           9957,  10400,  16798,   7515,  23046,  23611,      0],
        [ 23046,   6842,  10400,  22559,  19654,  13311,  14435,   7897,
          23046,   5358,  12713,  23046,   1770,      0,      0],
        [ 14044,   2709,  12713,  23046,  12791,    822,  23046,  23667,
          26444,   1943,  23046,  15355,      0,      0,      0],
        [ 17913,  21795,  13834,   1943,   8707,   5758,  12713,  23046,
          24993,   8956,  12713,   1263,      0,      0,      0],
      

In [15]:
print(cap_lens.size())
print(cap_lens)
# cap_lens_fake_input = torch.rand(48, dtype=torch.float64)
cap_lens_fake_input = torch.randint(1, 15, (48,), dtype=torch.long)
# cap_lens_dummy_input = torch.FloatTensor(cap_lens_dummy_input)
print(cap_lens_fake_input.size())
print(cap_lens_fake_input)

torch.Size([48])
tensor([ 15,  15,  14,  14,  13,  12,  12,  12,  11,  11,  10,  10,
         10,  10,  10,  10,  10,  10,  10,  10,   9,   9,   9,   9,
          9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,   9,
          9,   9,   9,   8,   8,   8,   8,   8,   8,   8,   8,   7])
torch.Size([48])
tensor([ 12,   6,  10,   8,   4,   7,   4,  13,   4,   9,  12,   9,
          5,   2,   1,  10,   8,  14,   7,   3,   2,  11,  11,  12,
         11,   2,   6,  10,  13,  12,  14,   5,   4,   1,   8,  10,
          5,   7,   5,   2,   4,   7,  14,  10,  12,  13,   2,   2])


In [16]:
print(hidden[1].shape)
print(hidden)
# hidden_fake_input = torch.randint(0, 1 (2, 48, 128), dtype=torch.long)
hidden_fake_input = torch.randint(1, 10, (2, 48, 128), dtype=torch.long)
# hidden_fake_input = torch.FloatTensor(hidden_fake_input)
print(hidden_fake_input.size())
print(hidden_fake_input)

torch.Size([2, 48, 128])
(tensor([[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]]]), tensor([[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  

In [17]:
# output = text_encoder(captions, cap_lens, hidden)
words_emb, sent_emb = text_encoder(captions, cap_lens, hidden)

r = (captions, cap_lens, hidden)

In [18]:
print(words_emb.shape)
print(words_emb)

torch.Size([48, 256, 15])
tensor([[[-0.0292, -0.0382, -0.0379,  ..., -0.0535, -0.0446, -0.0371],
         [ 0.0070,  0.0281,  0.0393,  ...,  0.0456,  0.0464,  0.0440],
         [ 0.0044,  0.0327,  0.0414,  ...,  0.0153,  0.0341,  0.0302],
         ...,
         [-0.0516, -0.0309, -0.0209,  ..., -0.0131, -0.0028, -0.0075],
         [ 0.0577,  0.0617,  0.0655,  ...,  0.0414,  0.0365,  0.0118],
         [ 0.0235,  0.0335,  0.0317,  ...,  0.0037,  0.0165, -0.0045]],

        [[-0.0292, -0.0368, -0.0577,  ..., -0.0468, -0.0493, -0.0409],
         [ 0.0070,  0.0246,  0.0302,  ...,  0.0160,  0.0187,  0.0059],
         [ 0.0044,  0.0238,  0.0317,  ...,  0.0286,  0.0343,  0.0372],
         ...,
         [-0.0365, -0.0062, -0.0123,  ..., -0.0086, -0.0003,  0.0005],
         [ 0.0601,  0.0671,  0.0671,  ...,  0.0326,  0.0300,  0.0079],
         [ 0.0156,  0.0182,  0.0367,  ...,  0.0258, -0.0126, -0.0019]],

        [[-0.0362, -0.0371, -0.0214,  ..., -0.0472, -0.0359,  0.0000],
         [ 0.0224, 

In [19]:
print(sent_emb.shape)
print(sent_emb)

torch.Size([48, 256])
tensor([[-3.7055e-02,  4.4027e-02,  3.0218e-02,  ..., -5.1560e-02,
          5.7713e-02,  2.3527e-02],
        [-4.0885e-02,  5.9492e-03,  3.7219e-02,  ..., -3.6472e-02,
          6.0148e-02,  1.5572e-02],
        [-3.5903e-02,  2.9381e-02,  4.0068e-02,  ..., -1.0184e-02,
          6.4992e-02,  3.9068e-02],
        ...,
        [-3.6976e-02,  3.3677e-02,  6.7880e-02,  ..., -2.8327e-02,
          4.7381e-02,  1.3940e-02],
        [-3.3234e-02,  1.4462e-02,  4.0952e-02,  ..., -1.0122e-03,
          7.6007e-02,  3.3161e-02],
        [-4.2610e-02,  9.3139e-03,  3.3283e-02,  ..., -4.0300e-02,
          4.7959e-02,  1.8901e-02]])


In [20]:
# source coreml/bin/activate
import coremltools

In [35]:
# output = text_encoder(captions, cap_lens, hidden)
captions_fake_input = torch.randint(1, 23046, (48, 15), dtype=torch.long)
cap_lens_fake_input = torch.randint(1, 15, (48,), dtype=torch.long)
hidden_fake_input = torch.randint(1, 10, (2, 48, 128), dtype=torch.long)

words_emb, sent_emb = text_encoder(captions_fake_input, cap_lens, hidden_fake_input)

TypeError: wrapPyFuncWithSymbolic(): incompatible function arguments. The following argument types are supported:
    1. (self: torch._C.Graph, arg0: function, arg1: List[torch::jit::Value], arg2: int, arg3: function) -> iterator

Invoked with: graph(%0 : Long(48, 15)
      %1 : Long(48)
      %2 : Float(2, 48, 128)
      %3 : Float(2, 48, 128)
      %4 : Float(27297, 300)
      %5 : Float(512, 300)
      %6 : Float(512, 128)
      %7 : Float(512)
      %8 : Float(512)
      %9 : Float(512, 300)
      %10 : Float(512, 128)
      %11 : Float(512)
      %12 : Float(512)) {
  %13 : Float(48, 15, 300) = aten::embedding[padding_idx=-1, scale_grad_by_freq=0, sparse=0](%4, %0), scope: RNN_ENCODER/Embedding[encoder]
  %16 : Float(48, 15, 300), %17 : Handle = ^Dropout(0.5, False, False)(%13), scope: RNN_ENCODER/Dropout[drop]
  %15 : Float(48, 15, 300) = aten::slice[dim=0, start=0, end=9223372036854775807, step=1](%13), scope: RNN_ENCODER/Dropout[drop]
  %14 : Float(48, 15, 300) = aten::as_strid

In [24]:
# Export the model
torch_out = torch.onnx._export(text_encoder,                 # model being run
                               (captions_fake_input, cap_lens, hidden), # model input (or a tuple for multiple inputs)
                               "super_resolution.onnx",      # where to save the model (can be a file or file-like object)
                               export_params=True)           # store the trained parameter weights inside the model file

TypeError: wrapPyFuncWithSymbolic(): incompatible function arguments. The following argument types are supported:
    1. (self: torch._C.Graph, arg0: function, arg1: List[torch::jit::Value], arg2: int, arg3: function) -> iterator

Invoked with: graph(%0 : Long(48, 15)
      %1 : Long(48)
      %2 : Float(2, 48, 128)
      %3 : Float(2, 48, 128)
      %4 : Float(27297, 300)
      %5 : Float(512, 300)
      %6 : Float(512, 128)
      %7 : Float(512)
      %8 : Float(512)
      %9 : Float(512, 300)
      %10 : Float(512, 128)
      %11 : Float(512)
      %12 : Float(512)) {
  %13 : Float(48, 15, 300) = aten::embedding[padding_idx=-1, scale_grad_by_freq=0, sparse=0](%4, %0), scope: RNN_ENCODER/Embedding[encoder]
  %16 : Float(48, 15, 300), %17 : Handle = ^Dropout(0.5, False, False)(%13), scope: RNN_ENCODER/Dropout[drop]
  %15 : Float(48, 15, 300) = aten::slice[dim=0, start=0, end=9223372036854775807, step=1](%13), scope: RNN_ENCODER/Dropout[drop]
  %14 : Float(48, 15, 300) = aten::as_strid